In [1]:
import argparse
import time
import csv
import pickle
import operator
import datetime
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from collections import Counter

In [3]:
df_rating = pd.read_csv('./dataset/ratings_Books.csv', sep=',', names=['UserId', 'itemID', 'rating', 'timestamp'])

In [4]:
df_rating

,UserId,itemID,rating,timestamp
0,AH2L9G3DQHHAJ,0000000116,4.0,1019865600
1,A2IIIDRK3PRRZY,0000000116,1.0,1395619200
2,A1TADCM7YWPQ8M,0000000868,4.0,1031702400
3,AWGH7V0BDOJKB,0000013714,4.0,1383177600
4,A3UTQPQPM4TQO0,0000013714,5.0,1374883200
...,...,...,...,...
22507150,A20KO0BPMNREJL,B00M13FNSS,5.0,1406073600
22507151,A1BQO66R6OLCCW,B00M13FNSS,5.0,1406073600
22507152,A1CI87FHKK6HVC,B00M13FNSS,5.0,1406073600
22507153,A8W4BR3HGGS3C,B00M13FNSS,5.0,1406073600


# Tmall

In [89]:
#創建知識圖譜的檔案
df_session = pd.read_csv('C:/Users/c8905/Downloads/Tmall/raw/dataset15.csv', sep='\t', names=['UserId', 'itemID', 'SessionId', 'timestamp'], skiprows=1)
df_kg = pd.read_csv('C:/Users/c8905/Downloads/Tmall/data_format1/user_log_format1.csv', sep=',', names=['UserId', 'itemID', 'cat_id', 'seller_id','brand_id','timestamp','action_type'], skiprows=1)

In [100]:
df_session = df_session[df_session['SessionId']<=120000]

df_session

,UserId,itemID,SessionId,timestamp
0,1,181459,0,1.444342e+09
1,1,779078,0,1.444342e+09
2,1,779078,0,1.444342e+09
3,1,452837,0,1.444342e+09
4,1,543397,0,1.444342e+09
...,...,...,...,...
818474,7675,986052,119999,1.435529e+09
818475,7675,953687,119999,1.435529e+09
818476,7675,724514,119999,1.435529e+09
818477,7675,199301,119999,1.435529e+09


In [101]:
#看每個session的長度
count_sess = Counter(df_session['SessionId'])
sess_count = pd.DataFrame({"class":count_sess.keys(),"count": count_sess.values()})
sess_count = sess_count[sess_count['count']>1]
sess_count = sess_count.reset_index(level=None, drop=True)
sess_count

,class,count
0,0,5
1,1,3
2,2,5
3,4,13
4,5,2
...,...,...
83376,119995,17
83377,119996,8
83378,119997,18
83379,119998,26


In [102]:
session = df_session[df_session['SessionId'].isin(sess_count['class'])]
session.reset_index(inplace=True, drop=True)
session

,UserId,itemID,SessionId,timestamp
0,1,181459,0,1.444342e+09
1,1,779078,0,1.444342e+09
2,1,779078,0,1.444342e+09
3,1,452837,0,1.444342e+09
4,1,543397,0,1.444342e+09
...,...,...,...,...
818474,7675,986052,119999,1.435529e+09
818475,7675,953687,119999,1.435529e+09
818476,7675,724514,119999,1.435529e+09
818477,7675,199301,119999,1.435529e+09


In [103]:
#看每個item出現的次數
count_item = Counter(session['itemID'])
item_count = pd.DataFrame({"class":count_item.keys(),"count": count_item.values()})

#保留出現次數大於五次的item
item_count = item_count[item_count['count']>=5]
item_count = item_count.reset_index(level=None, drop=True)
item_count

,class,count
0,779078,37
1,452837,5
2,504149,43
3,112203,67
4,556107,52
...,...,...
41507,535412,5
41508,396861,6
41509,741061,5
41510,320187,5


In [104]:
session = session[session['itemID'].isin(item_count['class'])]
session.reset_index(inplace=True, drop=True)

In [105]:
df_kg = df_kg[df_kg['itemID'].isin(item_count['class'])]
df_kg.reset_index(inplace=True, drop=True)
df_kg = df_kg.drop(columns=['UserId','timestamp','action_type'])

In [106]:
#因為會有重複的item，所以要去除
df_kg = df_kg.drop_duplicates(subset=['itemID'])
df_kg

,itemID,cat_id,seller_id,brand_id
0,239288,602,420,4953.0
1,950862,962,4605,7622.0
2,262500,602,420,4953.0
3,809148,602,420,4953.0
4,197239,602,420,4953.0
...,...,...,...,...
26874854,876957,737,1063,201.0
27040229,195837,1505,1579,7174.0
27390866,619015,252,2825,8245.0
27390868,940415,407,2825,8245.0


In [107]:
#如果資料集一開始是從0開始，需要加1
df_kg['cat_id'] = df_kg['cat_id']+max(df_kg['itemID'])
df_kg['seller_id'] = df_kg['seller_id']+max(df_kg['cat_id'])
df_kg['brand_id'] = df_kg['brand_id']+max(df_kg['seller_id'])
df_kg

,itemID,cat_id,seller_id,brand_id
0,239288,1113759,1115248,1124776.0
1,950862,1114119,1119433,1127445.0
2,262500,1113759,1115248,1124776.0
3,809148,1113759,1115248,1124776.0
4,197239,1113759,1115248,1124776.0
...,...,...,...,...
26874854,876957,1113894,1115891,1120024.0
27040229,195837,1114662,1116407,1126997.0
27390866,619015,1113409,1117653,1128068.0
27390868,940415,1113564,1117653,1128068.0


In [108]:
entity_encoder = LabelEncoder()

In [109]:
#entity_ID要從1開始
entity_encoder.fit(pd.concat([df_kg['itemID'], df_kg['cat_id'], df_kg['seller_id'], df_kg['brand_id']]))
df_kg['itemID'] = entity_encoder.transform(df_kg['itemID'])+1
df_kg['cat_id'] = entity_encoder.transform(df_kg['cat_id'])+1
df_kg['seller_id'] = entity_encoder.transform(df_kg['seller_id'])+1
df_kg['brand_id'] = entity_encoder.transform(df_kg['brand_id'])+1
df_kg

,itemID,cat_id,seller_id,brand_id
0,8944,41778,42599,49107
1,35447,41916,46309,50402
2,9790,41778,42599,49107
3,30169,41778,42599,49107
4,7345,41778,42599,49107
...,...,...,...,...
26874854,32704,41833,43178,46742
27040229,7303,42148,43630,50179
27390866,23123,41623,44737,50726
27390868,35085,41690,44737,50726


In [110]:
with open('kg.txt', "w") as f:
    for index, row in df_kg.iterrows():
        line = str(row['itemID'])+' 0 '+str(row['cat_id'])+'\n'
        line += str(row['itemID'])+' 1 '+str(row['seller_id'])+'\n'
        line += str(row['itemID'])+' 2 '+str(row['brand_id'])+'\n'
        f.write(line)

In [111]:
max(df_kg['brand_id'])

50841

In [79]:
session.to_csv("Tmall.csv",index=False)